In [1]:
import csv
import torch
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
import numpy as np

# hyperparameters
batch_size = 16  # how many independent sequences will we process in parallel?
block_size = 32  # what is the maximum context length for predictions?
max_iters = 1000  # step 的次數
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
vocab_size = 2000
# ------------

filepath = 'data/tx_data.csv'                               
filepath2 = 'data/tx_data0511.csv'

df = pd.read_csv(filepath)
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
print(df.head())

# 資料處理
torch.manual_seed(1337)

# 載入資料
df = pd.read_csv(filepath)
data_array = df.to_numpy().ravel()
data_array = data_array[(data_array >= 0)] - 14000
print(data_array , len(data_array))

# Train and test splits
# 分割training data (90%), val data (10%)
data = torch.tensor(data_array[:1000], dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)  # (B,T,C)
        q = self.query(x)  # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C ** -0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


# Transformer
class Transformer(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head

        self.sa = MultiHeadAttention(n_head, head_size)
        # self.encoder_layers = nn.ModuleList([EncoderLayer(n_head, head_size, n_embd, dropout) for _ in range(
        # num_layers)]) self.encoder_layers = nn.ModuleList([EncoderLayer(n_head, head_size, n_embd, dropout)])

        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Transformer(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # print(idx)
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)
        # tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters()) / 1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

FileNotFoundError: [Errno 2] No such file or directory: 'data/tx_data.csv'

In [8]:

# 帶入實際資料
df = pd.read_csv(filepath2)
test_array = df.to_numpy().ravel()
test_array = test_array[(test_array >= 0)]
input_test_array = test_array - 14000

context = torch.tensor([[input_test_array[0].astype(int)]])
for i in range(1, 8):
    context1 = torch.tensor([[input_test_array[i].astype(int)]])
    context = torch.cat((context, context1), dim=1)
    
print(context)
output = np.array(m.generate(context, max_new_tokens=100)[0].tolist())
output = output + 14000
print(output)

tensor([[1586, 1586, 1585, 1585, 1585, 1585, 1585, 1585]])
[15586 15586 15585 15585 15585 15585 15585 15585 15725 15725 15725 15725
 15725 15725 15725 15724 15724 15724 15726 15725 15724 15724 15724 15724
 15724 15724 15724 15723 15723 15723 15723 15723 15723 15723 15723 15723
 15723 15723 15723 15722 15722 15722 15722 15722 15722 15722 15722 15722
 15722 15722 15722 15722 15722 15723 15723 15723 15723 15723 15723 15724
 15722 15722 15722 15722 15722 15722 15723 15725 15723 15724 15724 15724
 15725 15723 15723 15723 15723 15723 15723 15723 15723 15724 15723 15724
 15724 15723 15724 15724 15725 15725 15725 15725 15725 15724 15724 15724
 15724 15725 15725 15725 15725 15726 15725 15725 15726 15725 15725 15725]


In [9]:
# Define the file name
file_name = 'output/data.csv'

# Save the NumPy array to a CSV file
np.savetxt(file_name, data_array, delimiter=',', fmt='%d')

In [10]:
filepath = 'data/tx_data0510.csv'
filepath2 = 'data/tx_data0511.csv'

df = pd.read_csv(filepath)
test_array = df.to_numpy().ravel()
test_array = test_array[(test_array >= 0)]

df = pd.read_csv(filepath)
output_array = df.to_numpy().ravel()
output_array = output_array[(output_array >= 0)]
# 假設你有兩組數據 data_array1 和 data_array2
data_array1 = test_array[:-1000]
data_array2 = output_array[:100] + 14000

# 創建 x 軸資料，這裡假設 x 軸是數列的索引
x = np.arange(len(data_array1))

# 繪製第一組數據（使用線條圖）
plt.plot(x, data_array1, label='act index', marker='o', linestyle='-')

# 繪製第二組數據（使用散點圖）
# plt.plot(x, data_array2, label='Transfomer index', marker='o', linestyle='-')
# plt.scatter(x, data_array2, label='Data 2', color='red')

# 加入標籤和標題
plt.xlabel('time')
plt.ylabel('Index')
plt.title('Multiple Data Plot')

# 加入圖例
plt.legend()

# 顯示圖表
plt.show()


NameError: name 'plt' is not defined